In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os

In [20]:
print("=" *100)
print(" 청와대 국민청원 게시판 크롤링하기 ")
print("=" *100)

query_txt = '청와대국민청원'

chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
url = 'https://www1.president.go.kr/petitions'
driver.get(url)
driver.maximize_window()

try :
    driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/table/tbody/tr/td[2]/div').click()
except :
    print('')
time.sleep(2)

print("\n")


collect_cnt = int(input('크롤링 할 건수는 몇건입니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 7)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print("\n")

f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\temp\\"

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

url_list=[]
count = 0

for a in range(1,collect_page_cnt+1) :
    
    print("%s 번째 페이지에서 게시글의 URL 정보를 추출합니다==============" %(a))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find_all('div','board text')
    result_2= result_1[2].find('div', class_='b_list category b_list2') 
    result_url_list=result_2.find('div','bl_body').find('ul','petition_list').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href']
        print(url)
        full_url = 'http://www1.president.go.kr/'+ url
        url_list.append(full_url)
               
        count += 1

        if count == collect_cnt :
            break  

    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(a,count))
    print("\n")
    
    a += 1
    try :
        driver.find_element_by_link_text("%s" %a).click()
    except :
        driver.find_element_by_link_text('Next').click()
        
    time.sleep(2)
        

no2=[]           # 게시글 번호 컬럼
title2=[]       # 게시글 제목 컬럼
peopl2=[]       # 참여인원
cate2=[]     # 카테고리
start2=[]        # 청원시작일
end2=[]       # 청원 종료일
content2=[]     # 청원 내용


no = 1

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','petitionsView_left_pg')

    f = open(ff_name, 'a',encoding='UTF-8')

    # 청원 등록 번호
    num = url_list[i].split('/')
    no_1 = num[5]        
    print('1.청원글번호: ',no_1 ,'\n')
    f.write('\n')
    f.write("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no + '\n')
    f.write('1.청원번호:' + no_1 + '\n')
    no2.append(no_1)
    
    # 청원제목 추출
    title = content_all.find('h3','petitionsView_title').get_text()
    print('2.청원제목: ' , title , '\n')
    f.write('2.청원제목:' + title + '\n')
    title2.append(title)
    
    # 참여인원 추출
    peopl = content_all.find('h2','petitionsView_count').find('span','counter').get_text()
    print('3.참여인원: ' , peopl ,'명' ,'\n')
    f.write('3.참여인원:' + peopl + '\n')
    peopl2.append(peopl)
    
    # 카테고리
    cate_all = content_all.find('div','petitionsView_info').find('ul','petitionsView_info_list').find_all('li')

    category = cate_all[0].get_text().replace('카테고리','')
    print('4.카테고리: ',category ,'\n')
    f.write('4.카테고리:' + category + '\n')
    cate2.append(category)
    
    #청원시작일
    start = cate_all[1].get_text().replace('청원시작','')
    print('5.청원시작일: ',start ,'\n')
    f.write('5.청원시작일:' + start + '\n')
    start2.append(category)
    
    #청원마감일
    end = cate_all[2].get_text().replace('청원마감','')
    print('6.청원마감일: ',end ,'\n')
    f.write('6.청원마감일:' + end + '\n')
    end2.append(category)
    
    #청원내용
    content = content_all.find('div','View_write').get_text()
    print('7.청원내용: ',content ,'\n')
    f.write('7.청원내용:' + content + '\n')
    content2.append(category)
    
    f.close( )

    no+=1

    if no > collect_cnt :
        break
        

blue_house = pd.DataFrame()

blue_house['청원번호'] = no2
blue_house['청원제목'] = pd.Series(title2) 
blue_house['참여자'] = pd.Series(peopl2)
blue_house['카테고리'] = pd.Series(cate2)
blue_house['청원시작일'] = pd.Series(start2)
blue_house['청원종료일'] = pd.Series(end2)
blue_house['청원내용'] = pd.Series(content2)
       
# csv 형태로 저장하기
blue_house.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
blue_house.to_excel(fx_name , index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

#Step 8. 요약정보 출력하기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )

 청와대 국민청원 게시판 크롤링하기 


크롤링 할 건수는 몇건입니까?: 3
3 건의 데이터를 수집하기 위해 1 페이지의 게시물을 조회합니다.


2.파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):
1 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/599809
/petitions/599808
/petitions/599807
1 번째 페이지까지 3건 정보 수집 완료 ====================




1 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  599809 

2.청원제목:  (얀센) 형부_백신접종 후 사지마비, 결국 중환자실 

3.참여인원:  1,861 명 

4.카테고리:  보건복지 

5.청원시작일:  2021-07-13 

6.청원마감일:  2021-08-12 

7.청원내용:  
										(얀센) 백신접종 후 사지마비, 결국 중환자실 

현재 이 상황에 처한 언니는 이루 말할 수 없는 충격에 이 글을 작성할 수 없기에 동생인 제가 대신 적어보려 합니다. 

저희 형부가 얀센백신 접종(2021년 06월 18일)후 약 10일후부터 이상반응이 오기 시작했습니다. 그 전까지는 약간의 피곤함 무기력함 등 거의 무증상으로 일상생활이 가능했고 늘상 해오던 본인의 업도 수행 가능할 만큼 건강 했습니다. 특이 기저질환 및 병력도 없었을 뿐더러, 키는 무려 187㎝에 몸무게 90kg의 건강한 체구로, 덩치 큰 두 아들(5살, 8살)을 두 팔로 동시에 번쩍번쩍 들 만큼 건강하고 건장 했던 40대 초반의 남자였습니다. 

문제는 접종 후 10일 이후(21년 06월 28일)부터 였다고 합니다. 오전부터 손발 저리는 증상 발생과 함께 깨질 듯 한 두통이 시작되었지만 그래도 이때는 그나마 완전한 보행이 가능했습니다. 

이틀 후(21년 6월 30일부터) 상태는 더욱 악화 되었습니다. 부축하면 보행이 가능하나 두통이 너무 심했고, 타이레놀을 먹어

In [4]:
now = time.localtime()
print(now)

time.struct_time(tm_year=2021, tm_mon=7, tm_mday=14, tm_hour=19, tm_min=23, tm_sec=38, tm_wday=2, tm_yday=195, tm_isdst=0)
